## EMA 与 MACD 配合使用

MACD 是基于 EMA 的动量指标，由快线（MACD 线）、慢线（信号线）和柱状图（Histogram）组成。

#### 使用方法：
+ 当 MACD 线（快线）上穿信号线（慢线）时，可能是买入信号。
+ 当 MACD 线下穿信号线时，可能是卖出信号。
结合 EMA（如 12 日和 26 日 EMA），可以进一步确认趋势方向。

In [ ]:
# 获取数据源
import baostock as bs
import pandas as pd
import logging

def get_k_data(self,code, start, end):
    lg = bs.login()
    # 显示登陆返回信息
    logging.info('login respond error_code:' + lg.error_code)
    logging.info('login respond  error_msg:' + lg.error_msg)
    # 获取股票数据
    rs = bs.query_history_k_data_plus(code,
                                              "date,open,high,low,close,volume",
                                              start_date=start,
                                              end_date=end,
                                              frequency="d",
                                              adjustflag="3")  # adjustflag="3"表示不复权
    data = rs.get_data()


    # 登出baostock系统
    bs.logout()

    # 创建DataFrame
    df = data[['date', 'open', 'high', 'low', 'close', 'volume']].rename(columns={'date': 'Date'})

    # 转换日期格式
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)  # 设置日期列为索引

    # 将空字符串替换为 NaN
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].replace('', np.nan)
    # 去除包含空值的行
    df = df.dropna(subset=['open', 'high', 'low', 'close', 'volume'])

    # 将股价数据转换为数值类型
    df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df


In [ ]:
stock_code = "sh.688027"  # 沪深300
start_date = "2023-01-01"
end_date = "2024-12-31"

df = b.get_k_data(stock_code, start_date, end_date)